# Week 14 - Day 6: Practical Trading with RNNs

## Learning Objectives
- Implement **sequence classification** for market regime detection
- Understand **many-to-one vs many-to-many** RNN architectures
- Combine **LSTM with technical indicators** for enhanced predictions
- Build a **complete trading strategy with backtesting**

---

## Table of Contents
1. [Setup and Data Preparation](#1-setup)
2. [Sequence Classification for Regime Detection](#2-regime)
3. [Many-to-One vs Many-to-Many Architectures](#3-architectures)
4. [LSTM + Technical Indicators Combination](#4-lstm-indicators)
5. [Full Trading Strategy with Backtest](#5-backtest)
6. [Summary and Key Takeaways](#6-summary)

---
<a id='1-setup'></a>
## 1. Setup and Data Preparation

First, let's import all necessary libraries and download market data.

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Data
import yfinance as yf

# Sklearn utilities
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Set seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

In [ ]:
# Download market data
tickers = ['SPY', 'QQQ', 'IWM']  # S&P 500, Nasdaq, Russell 2000
start_date = '2015-01-01'
end_date = '2024-01-01'

# Download data
data = {}
for ticker in tickers:
    df = yf.download(ticker, start=start_date, end=end_date, progress=False)
    data[ticker] = df['Close']

# Create DataFrame
prices = pd.DataFrame(data)
prices.columns = prices.columns.droplevel(1) if isinstance(prices.columns, pd.MultiIndex) else prices.columns
prices = prices.dropna()

print(f"Data shape: {prices.shape}")
print(f"Date range: {prices.index[0]} to {prices.index[-1]}")
prices.head()

In [ ]:
# Calculate returns
returns = prices.pct_change().dropna()

# Focus on SPY for main analysis
spy_close = prices['SPY'].values
spy_returns = returns['SPY'].values

# Visualize price and returns
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(prices.index, prices['SPY'], label='SPY', color='blue')
axes[0].set_title('SPY Close Price', fontsize=14)
axes[0].set_ylabel('Price ($)')
axes[0].legend()

axes[1].plot(returns.index, returns['SPY'], label='Returns', color='green', alpha=0.7)
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1].set_title('SPY Daily Returns', fontsize=14)
axes[1].set_ylabel('Return')
axes[1].legend()

plt.tight_layout()
plt.show()

---
<a id='2-regime'></a>
## 2. Sequence Classification for Regime Detection

### What is Regime Detection?

Market regimes represent distinct market states characterized by:
- **Bull Market**: Sustained upward trend, low volatility
- **Bear Market**: Sustained downward trend, high volatility
- **Sideways/Consolidation**: Range-bound, moderate volatility
- **High Volatility**: Large swings regardless of direction

We'll use an LSTM to classify market regimes based on historical price sequences.

In [ ]:
def calculate_regime_labels(prices_series, lookback=20):
    """
    Create regime labels based on rolling statistics.
    
    Regimes:
    0 - Bear (negative return, high vol)
    1 - Consolidation (low abs return, low vol)
    2 - Bull (positive return, moderate vol)
    3 - High Volatility (high vol regardless of direction)
    """
    df = pd.DataFrame(index=prices_series.index)
    df['price'] = prices_series.values
    df['returns'] = df['price'].pct_change()
    
    # Rolling statistics
    df['rolling_return'] = df['returns'].rolling(lookback).sum()
    df['rolling_vol'] = df['returns'].rolling(lookback).std() * np.sqrt(252)
    
    # Percentile thresholds
    vol_median = df['rolling_vol'].median()
    vol_75 = df['rolling_vol'].quantile(0.75)
    
    # Classify regimes
    def classify_regime(row):
        if pd.isna(row['rolling_vol']):
            return np.nan
        
        if row['rolling_vol'] > vol_75:
            return 3  # High Volatility
        elif row['rolling_return'] < -0.02 and row['rolling_vol'] > vol_median:
            return 0  # Bear
        elif row['rolling_return'] > 0.02:
            return 2  # Bull
        else:
            return 1  # Consolidation
    
    df['regime'] = df.apply(classify_regime, axis=1)
    
    return df.dropna()

# Calculate regimes
regime_df = calculate_regime_labels(prices['SPY'])
print("Regime Distribution:")
print(regime_df['regime'].value_counts().sort_index())
print("\nRegime Labels: 0=Bear, 1=Consolidation, 2=Bull, 3=High Volatility")

In [ ]:
# Visualize regimes
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot price with regime colors
regime_colors = {0: 'red', 1: 'gray', 2: 'green', 3: 'orange'}
regime_names = {0: 'Bear', 1: 'Consolidation', 2: 'Bull', 3: 'High Volatility'}

axes[0].plot(regime_df.index, regime_df['price'], color='black', alpha=0.5, linewidth=0.5)
for regime in [0, 1, 2, 3]:
    mask = regime_df['regime'] == regime
    axes[0].scatter(regime_df.index[mask], regime_df['price'][mask], 
                   c=regime_colors[regime], label=regime_names[regime], s=5, alpha=0.6)
axes[0].set_title('SPY Price Colored by Market Regime', fontsize=14)
axes[0].set_ylabel('Price ($)')
axes[0].legend(loc='upper left')

# Regime over time
axes[1].plot(regime_df.index, regime_df['regime'], color='blue', alpha=0.7)
axes[1].set_title('Market Regime Over Time', fontsize=14)
axes[1].set_ylabel('Regime')
axes[1].set_yticks([0, 1, 2, 3])
axes[1].set_yticklabels(['Bear', 'Consolidation', 'Bull', 'High Vol'])

plt.tight_layout()
plt.show()

In [ ]:
class RegimeClassifierLSTM(nn.Module):
    """
    LSTM for Sequence Classification (Many-to-One)
    
    Takes a sequence of features and outputs a single class prediction.
    """
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.2):
        super(RegimeClassifierLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=False
        )
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, num_classes)
        )
        
    def forward(self, x):
        # x shape: (batch, seq_len, input_size)
        
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # LSTM forward pass
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        # Take the last hidden state (many-to-one)
        out = out[:, -1, :]  # Shape: (batch, hidden_size)
        
        # Classification
        out = self.fc(out)
        
        return out

In [ ]:
def prepare_regime_data(df, seq_length=30, train_split=0.8):
    """
    Prepare sequences for regime classification.
    """
    # Features: returns, rolling volatility, rolling return
    features = df[['returns', 'rolling_vol', 'rolling_return']].values
    labels = df['regime'].values.astype(int)
    
    # Scale features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Create sequences
    X, y = [], []
    for i in range(seq_length, len(features_scaled)):
        X.append(features_scaled[i-seq_length:i])
        y.append(labels[i])
    
    X = np.array(X)
    y = np.array(y)
    
    # Train/test split
    split_idx = int(len(X) * train_split)
    
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    # Convert to tensors
    X_train = torch.FloatTensor(X_train)
    X_test = torch.FloatTensor(X_test)
    y_train = torch.LongTensor(y_train)
    y_test = torch.LongTensor(y_test)
    
    return X_train, X_test, y_train, y_test, scaler

# Prepare data
SEQ_LENGTH = 30
X_train, X_test, y_train, y_test, scaler = prepare_regime_data(regime_df, SEQ_LENGTH)

print(f"Training set: X={X_train.shape}, y={y_train.shape}")
print(f"Test set: X={X_test.shape}, y={y_test.shape}")

In [ ]:
# Create data loaders
BATCH_SIZE = 64

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize model
INPUT_SIZE = 3  # Number of features
HIDDEN_SIZE = 64
NUM_LAYERS = 2
NUM_CLASSES = 4  # 4 regimes

model_regime = RegimeClassifierLSTM(
    input_size=INPUT_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    num_classes=NUM_CLASSES
).to(device)

print(model_regime)

In [ ]:
def train_classifier(model, train_loader, test_loader, epochs=50, lr=0.001):
    """
    Train the regime classifier.
    """
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    
    train_losses = []
    test_losses = []
    test_accuracies = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        # Evaluation
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                test_loss += loss.item()
                
                _, predicted = torch.max(outputs, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()
        
        test_loss /= len(test_loader)
        test_losses.append(test_loss)
        accuracy = correct / total
        test_accuracies.append(accuracy)
        
        scheduler.step(test_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, '
                  f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')
    
    return train_losses, test_losses, test_accuracies

# Train the model
train_losses, test_losses, test_accuracies = train_classifier(
    model_regime, train_loader, test_loader, epochs=50
)

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(train_losses, label='Train Loss')
axes[0].plot(test_losses, label='Test Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Test Loss')
axes[0].legend()

axes[1].plot(test_accuracies, label='Test Accuracy', color='green')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Test Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate on test set
model_regime.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model_regime(X_batch)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(y_batch.numpy())

# Classification report
print("Classification Report:")
print(classification_report(all_labels, all_preds, 
                           target_names=['Bear', 'Consolidation', 'Bull', 'High Vol']))

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Bear', 'Consol', 'Bull', 'HiVol'],
            yticklabels=['Bear', 'Consol', 'Bull', 'HiVol'])
plt.title('Regime Classification Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

---
<a id='3-architectures'></a>
## 3. Many-to-One vs Many-to-Many Architectures

### Architecture Comparison

| Architecture | Input | Output | Use Case |
|-------------|-------|--------|----------|
| **Many-to-One** | Sequence | Single value | Classification, next-day prediction |
| **Many-to-Many** | Sequence | Sequence | Multi-step forecasting, sequence labeling |

![RNN Architectures](https://miro.medium.com/max/1400/1*6xj691fPWD8ns3LtmJCNvA.jpeg)

In [ ]:
class ManyToOneLSTM(nn.Module):
    """
    Many-to-One: Predict single output from sequence.
    Used for: next-day direction, regime classification
    """
    def __init__(self, input_size, hidden_size, num_layers=2, output_size=1):
        super(ManyToOneLSTM, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2
        )
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # x: (batch, seq_len, input_size)
        lstm_out, (hn, cn) = self.lstm(x)
        # Take ONLY the last output
        out = self.fc(lstm_out[:, -1, :])  # (batch, output_size)
        return out


class ManyToManyLSTM(nn.Module):
    """
    Many-to-Many: Predict sequence from sequence.
    Used for: multi-step forecasting, sequence tagging
    """
    def __init__(self, input_size, hidden_size, num_layers=2, output_size=1, forecast_horizon=5):
        super(ManyToManyLSTM, self).__init__()
        
        self.forecast_horizon = forecast_horizon
        
        # Encoder LSTM
        self.encoder = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2
        )
        
        # Decoder LSTM
        self.decoder = nn.LSTM(
            input_size=output_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2
        )
        
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, target=None, teacher_forcing_ratio=0.5):
        batch_size = x.size(0)
        
        # Encode input sequence
        _, (hidden, cell) = self.encoder(x)
        
        # Initialize decoder input with last value
        decoder_input = x[:, -1, :1].unsqueeze(1)  # (batch, 1, 1)
        
        outputs = []
        
        for t in range(self.forecast_horizon):
            decoder_out, (hidden, cell) = self.decoder(decoder_input, (hidden, cell))
            prediction = self.fc(decoder_out[:, -1, :])  # (batch, output_size)
            outputs.append(prediction)
            
            # Teacher forcing
            if target is not None and np.random.random() < teacher_forcing_ratio:
                decoder_input = target[:, t:t+1, :]  # Use actual value
            else:
                decoder_input = prediction.unsqueeze(1)  # Use prediction
        
        outputs = torch.stack(outputs, dim=1)  # (batch, forecast_horizon, output_size)
        return outputs


print("Many-to-One Model:")
m2o = ManyToOneLSTM(input_size=3, hidden_size=64, output_size=1)
print(m2o)
print(f"\nOutput shape for input (32, 30, 3): {m2o(torch.randn(32, 30, 3)).shape}")

print("\n" + "="*60)
print("\nMany-to-Many Model:")
m2m = ManyToManyLSTM(input_size=3, hidden_size=64, output_size=1, forecast_horizon=5)
print(m2m)
print(f"\nOutput shape for input (32, 30, 3): {m2m(torch.randn(32, 30, 3)).shape}")

In [ ]:
def prepare_multi_step_data(prices_series, seq_length=30, forecast_horizon=5, train_split=0.8):
    """
    Prepare data for many-to-many forecasting.
    """
    # Calculate returns
    returns = prices_series.pct_change().dropna().values.reshape(-1, 1)
    
    # Scale
    scaler = StandardScaler()
    returns_scaled = scaler.fit_transform(returns)
    
    # Create sequences
    X, y = [], []
    for i in range(seq_length, len(returns_scaled) - forecast_horizon):
        X.append(returns_scaled[i-seq_length:i])
        y.append(returns_scaled[i:i+forecast_horizon])
    
    X = np.array(X)
    y = np.array(y)
    
    # Split
    split_idx = int(len(X) * train_split)
    
    X_train = torch.FloatTensor(X[:split_idx])
    X_test = torch.FloatTensor(X[split_idx:])
    y_train = torch.FloatTensor(y[:split_idx])
    y_test = torch.FloatTensor(y[split_idx:])
    
    return X_train, X_test, y_train, y_test, scaler

# Prepare multi-step data
FORECAST_HORIZON = 5
X_train_ms, X_test_ms, y_train_ms, y_test_ms, scaler_ms = prepare_multi_step_data(
    prices['SPY'], seq_length=30, forecast_horizon=FORECAST_HORIZON
)

print(f"Many-to-Many Data:")
print(f"X_train: {X_train_ms.shape}, y_train: {y_train_ms.shape}")
print(f"X_test: {X_test_ms.shape}, y_test: {y_test_ms.shape}")

In [ ]:
# Train Many-to-Many model for multi-step forecasting
model_m2m = ManyToManyLSTM(
    input_size=1, 
    hidden_size=64, 
    output_size=1, 
    forecast_horizon=FORECAST_HORIZON
).to(device)

criterion_m2m = nn.MSELoss()
optimizer_m2m = optim.Adam(model_m2m.parameters(), lr=0.001)

train_dataset_ms = TensorDataset(X_train_ms, y_train_ms)
train_loader_ms = DataLoader(train_dataset_ms, batch_size=64, shuffle=True)

# Training loop
epochs = 30
losses = []

for epoch in range(epochs):
    model_m2m.train()
    epoch_loss = 0
    
    for X_batch, y_batch in train_loader_ms:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer_m2m.zero_grad()
        
        # Teacher forcing ratio decays over epochs
        tf_ratio = max(0.1, 1 - epoch / epochs)
        outputs = model_m2m(X_batch, y_batch, teacher_forcing_ratio=tf_ratio)
        
        loss = criterion_m2m(outputs, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_m2m.parameters(), max_norm=1.0)
        optimizer_m2m.step()
        
        epoch_loss += loss.item()
    
    losses.append(epoch_loss / len(train_loader_ms))
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {losses[-1]:.6f}')

plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.title('Many-to-Many Model Training Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.show()

In [ ]:
# Visualize multi-step predictions
model_m2m.eval()

# Get predictions for a sample
sample_idx = 100
X_sample = X_test_ms[sample_idx:sample_idx+1].to(device)
y_actual = y_test_ms[sample_idx].numpy().flatten()

with torch.no_grad():
    y_pred = model_m2m(X_sample).cpu().numpy().flatten()

# Plot
fig, ax = plt.subplots(figsize=(12, 5))

# Historical data
hist = X_sample.cpu().numpy().flatten()
ax.plot(range(len(hist)), hist, 'b-', label='Historical', linewidth=2)

# Forecasts
forecast_x = range(len(hist), len(hist) + FORECAST_HORIZON)
ax.plot(forecast_x, y_actual, 'g-o', label='Actual', linewidth=2, markersize=8)
ax.plot(forecast_x, y_pred, 'r--o', label='Predicted', linewidth=2, markersize=8)

ax.axvline(x=len(hist)-1, color='gray', linestyle='--', alpha=0.5)
ax.set_title('Many-to-Many: 5-Day Return Forecast', fontsize=14)
ax.set_xlabel('Time Step')
ax.set_ylabel('Scaled Return')
ax.legend()
plt.tight_layout()
plt.show()

---
<a id='4-lstm-indicators'></a>
## 4. LSTM + Technical Indicators Combination

Combining LSTM with technical indicators can improve predictions by:
1. **Adding domain knowledge** - Technical indicators capture known patterns
2. **Feature engineering** - Pre-computed signals reduce what LSTM must learn
3. **Multi-scale information** - Indicators at different timeframes

In [ ]:
def calculate_technical_indicators(prices_df, ticker='SPY'):
    """
    Calculate technical indicators for LSTM input.
    """
    df = pd.DataFrame(index=prices_df.index)
    close = prices_df[ticker]
    
    # Returns
    df['returns'] = close.pct_change()
    df['log_returns'] = np.log(close / close.shift(1))
    
    # Moving averages
    df['sma_10'] = close.rolling(10).mean() / close - 1
    df['sma_20'] = close.rolling(20).mean() / close - 1
    df['sma_50'] = close.rolling(50).mean() / close - 1
    
    # EMA
    df['ema_12'] = close.ewm(span=12).mean() / close - 1
    df['ema_26'] = close.ewm(span=26).mean() / close - 1
    
    # MACD
    ema12 = close.ewm(span=12).mean()
    ema26 = close.ewm(span=26).mean()
    df['macd'] = (ema12 - ema26) / close
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # RSI
    delta = close.diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    df['rsi'] = (100 - (100 / (1 + rs))) / 100 - 0.5  # Normalize around 0
    
    # Bollinger Bands
    sma20 = close.rolling(20).mean()
    std20 = close.rolling(20).std()
    df['bb_upper'] = (sma20 + 2 * std20) / close - 1
    df['bb_lower'] = (sma20 - 2 * std20) / close - 1
    df['bb_width'] = (df['bb_upper'] - df['bb_lower'])
    df['bb_position'] = (close - (sma20 - 2*std20)) / (4 * std20) - 0.5
    
    # Volatility
    df['volatility_10'] = df['returns'].rolling(10).std() * np.sqrt(252)
    df['volatility_20'] = df['returns'].rolling(20).std() * np.sqrt(252)
    
    # Momentum
    df['momentum_5'] = close.pct_change(5)
    df['momentum_10'] = close.pct_change(10)
    df['momentum_20'] = close.pct_change(20)
    
    # Target: next day return direction (1 for up, 0 for down)
    df['target'] = (df['returns'].shift(-1) > 0).astype(int)
    
    return df.dropna()

# Calculate indicators
indicators_df = calculate_technical_indicators(prices)
print(f"Features: {indicators_df.columns.tolist()}")
print(f"\nShape: {indicators_df.shape}")
indicators_df.head()

In [ ]:
# Visualize some indicators
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Price with SMAs
ax0 = axes[0]
ax0.plot(prices.index[-500:], prices['SPY'].values[-500:], label='SPY', alpha=0.8)
ax0.set_title('SPY Price (Last 500 Days)')
ax0.legend()

# MACD
ax1 = axes[1]
ax1.plot(indicators_df.index[-500:], indicators_df['macd'].values[-500:], label='MACD', color='blue')
ax1.plot(indicators_df.index[-500:], indicators_df['macd_signal'].values[-500:], label='Signal', color='orange')
ax1.bar(indicators_df.index[-500:], indicators_df['macd_hist'].values[-500:], alpha=0.3, label='Histogram')
ax1.axhline(y=0, color='gray', linestyle='--')
ax1.set_title('MACD')
ax1.legend()

# RSI
ax2 = axes[2]
rsi_plot = (indicators_df['rsi'] + 0.5) * 100  # Convert back to 0-100 scale
ax2.plot(indicators_df.index[-500:], rsi_plot.values[-500:], color='purple')
ax2.axhline(y=70, color='red', linestyle='--', alpha=0.5)
ax2.axhline(y=30, color='green', linestyle='--', alpha=0.5)
ax2.fill_between(indicators_df.index[-500:], 30, 70, alpha=0.1)
ax2.set_title('RSI')
ax2.set_ylim(0, 100)

# Volatility
ax3 = axes[3]
ax3.plot(indicators_df.index[-500:], indicators_df['volatility_20'].values[-500:], color='red')
ax3.set_title('20-Day Rolling Volatility (Annualized)')
ax3.set_ylabel('Volatility')

plt.tight_layout()
plt.show()

In [ ]:
class LSTM_TechnicalIndicators(nn.Module):
    """
    LSTM with Technical Indicators for Trading Signal Generation.
    
    Architecture:
    - Bidirectional LSTM for pattern recognition
    - Attention mechanism for important timesteps
    - Dense layers for classification
    """
    def __init__(self, input_size, hidden_size, num_layers=2, dropout=0.3):
        super(LSTM_TechnicalIndicators, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        # x: (batch, seq_len, input_size)
        
        # LSTM
        lstm_out, _ = self.lstm(x)  # (batch, seq_len, hidden*2)
        
        # Attention
        attention_weights = self.attention(lstm_out)  # (batch, seq_len, 1)
        attention_weights = torch.softmax(attention_weights, dim=1)
        
        # Weighted sum
        context = torch.sum(attention_weights * lstm_out, dim=1)  # (batch, hidden*2)
        
        # Classification
        output = self.classifier(context)
        
        return output.squeeze(-1), attention_weights.squeeze(-1)

In [ ]:
def prepare_indicator_data(df, seq_length=20, train_split=0.8):
    """
    Prepare technical indicator data for LSTM.
    """
    # Feature columns (exclude target)
    feature_cols = [col for col in df.columns if col != 'target']
    
    features = df[feature_cols].values
    targets = df['target'].values
    
    # Scale features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Create sequences
    X, y = [], []
    for i in range(seq_length, len(features_scaled)):
        X.append(features_scaled[i-seq_length:i])
        y.append(targets[i])
    
    X = np.array(X)
    y = np.array(y)
    
    # Split
    split_idx = int(len(X) * train_split)
    
    X_train = torch.FloatTensor(X[:split_idx])
    X_test = torch.FloatTensor(X[split_idx:])
    y_train = torch.FloatTensor(y[:split_idx])
    y_test = torch.FloatTensor(y[split_idx:])
    
    return X_train, X_test, y_train, y_test, scaler, feature_cols

# Prepare data
X_train_ind, X_test_ind, y_train_ind, y_test_ind, scaler_ind, feature_cols = prepare_indicator_data(
    indicators_df, seq_length=20
)

print(f"Training set: X={X_train_ind.shape}, y={y_train_ind.shape}")
print(f"Test set: X={X_test_ind.shape}, y={y_test_ind.shape}")
print(f"\nFeatures ({len(feature_cols)}): {feature_cols}")

In [ ]:
# Create model and train
model_indicators = LSTM_TechnicalIndicators(
    input_size=len(feature_cols),
    hidden_size=64,
    num_layers=2,
    dropout=0.3
).to(device)

criterion_ind = nn.BCELoss()
optimizer_ind = optim.Adam(model_indicators.parameters(), lr=0.001)
scheduler_ind = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ind, patience=5, factor=0.5)

# Data loaders
train_dataset_ind = TensorDataset(X_train_ind, y_train_ind)
test_dataset_ind = TensorDataset(X_test_ind, y_test_ind)
train_loader_ind = DataLoader(train_dataset_ind, batch_size=64, shuffle=True)
test_loader_ind = DataLoader(test_dataset_ind, batch_size=64, shuffle=False)

# Training
epochs = 50
train_losses_ind = []
test_accuracies_ind = []

for epoch in range(epochs):
    # Training
    model_indicators.train()
    train_loss = 0
    
    for X_batch, y_batch in train_loader_ind:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer_ind.zero_grad()
        outputs, _ = model_indicators(X_batch)
        loss = criterion_ind(outputs, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_indicators.parameters(), max_norm=1.0)
        optimizer_ind.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader_ind)
    train_losses_ind.append(train_loss)
    
    # Evaluation
    model_indicators.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader_ind:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs, _ = model_indicators(X_batch)
            predicted = (outputs > 0.5).float()
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    
    accuracy = correct / total
    test_accuracies_ind.append(accuracy)
    scheduler_ind.step(train_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_loss:.4f}, Accuracy: {accuracy:.4f}')

In [ ]:
# Visualize attention weights
model_indicators.eval()

# Get attention for a sample
sample_idx = 50
X_sample = X_test_ind[sample_idx:sample_idx+1].to(device)

with torch.no_grad():
    pred, attention = model_indicators(X_sample)

attention = attention.cpu().numpy().flatten()

# Plot
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Attention weights
axes[0].bar(range(len(attention)), attention, color='blue', alpha=0.7)
axes[0].set_xlabel('Time Step')
axes[0].set_ylabel('Attention Weight')
axes[0].set_title('Attention Weights Across Sequence')

# Feature values with attention overlay
sample_data = X_sample.cpu().numpy()[0]
im = axes[1].imshow(sample_data.T, aspect='auto', cmap='RdYlGn')
axes[1].set_xlabel('Time Step')
axes[1].set_ylabel('Feature')
axes[1].set_yticks(range(len(feature_cols)))
axes[1].set_yticklabels(feature_cols, fontsize=8)
axes[1].set_title('Feature Values Over Sequence (with Attention)')

# Overlay attention
ax2 = axes[1].twinx()
ax2.plot(range(len(attention)), attention * len(feature_cols), 'k-', linewidth=2, label='Attention')
ax2.set_ylabel('Attention')

plt.colorbar(im, ax=axes[1], label='Feature Value')
plt.tight_layout()
plt.show()

---
<a id='5-backtest'></a>
## 5. Full Trading Strategy with Backtest

Now let's build a complete trading system that:
1. Uses the LSTM + Technical Indicators model for signal generation
2. Implements position sizing based on prediction confidence
3. Includes transaction costs
4. Calculates comprehensive performance metrics

In [ ]:
class TradingBacktester:
    """
    Backtesting framework for LSTM trading strategies.
    """
    def __init__(self, initial_capital=100000, transaction_cost=0.001):
        self.initial_capital = initial_capital
        self.transaction_cost = transaction_cost
        
    def generate_signals(self, model, X_data, threshold=0.5):
        """
        Generate trading signals from model predictions.
        
        Returns:
            signals: 1 (long), 0 (no position), -1 (short)
            probabilities: raw model outputs
        """
        model.eval()
        
        with torch.no_grad():
            X_tensor = X_data.to(device)
            probs, _ = model(X_tensor)
            probs = probs.cpu().numpy()
        
        # Generate signals based on probability threshold
        signals = np.zeros(len(probs))
        signals[probs > 0.5 + threshold/2] = 1   # Strong buy
        signals[probs < 0.5 - threshold/2] = -1  # Strong sell
        
        return signals, probs
    
    def run_backtest(self, signals, returns, probabilities=None):
        """
        Run backtest with given signals.
        
        Args:
            signals: trading signals (1, 0, -1)
            returns: actual returns for the period
            probabilities: model confidence for position sizing
        """
        n = len(signals)
        
        # Position sizing based on confidence
        if probabilities is not None:
            confidence = np.abs(probabilities - 0.5) * 2  # Scale to 0-1
            positions = signals * confidence
        else:
            positions = signals
        
        # Calculate strategy returns
        strategy_returns = positions[:-1] * returns[1:]
        
        # Transaction costs (when position changes)
        position_changes = np.abs(np.diff(positions))
        costs = position_changes * self.transaction_cost
        strategy_returns = strategy_returns - costs
        
        # Calculate equity curve
        equity = self.initial_capital * np.cumprod(1 + strategy_returns)
        equity = np.insert(equity, 0, self.initial_capital)
        
        # Buy and hold benchmark
        benchmark_equity = self.initial_capital * np.cumprod(1 + returns)
        benchmark_equity = np.insert(benchmark_equity, 0, self.initial_capital)
        
        results = {
            'signals': signals,
            'positions': positions,
            'strategy_returns': strategy_returns,
            'equity': equity,
            'benchmark_equity': benchmark_equity[:len(equity)],
            'returns': returns
        }
        
        return results
    
    def calculate_metrics(self, results):
        """
        Calculate comprehensive performance metrics.
        """
        strategy_returns = results['strategy_returns']
        equity = results['equity']
        benchmark_equity = results['benchmark_equity']
        
        # Total returns
        total_return = (equity[-1] / equity[0]) - 1
        benchmark_return = (benchmark_equity[-1] / benchmark_equity[0]) - 1
        
        # Annualized returns (assuming 252 trading days)
        n_days = len(strategy_returns)
        annual_return = (1 + total_return) ** (252 / n_days) - 1
        benchmark_annual = (1 + benchmark_return) ** (252 / n_days) - 1
        
        # Volatility
        volatility = np.std(strategy_returns) * np.sqrt(252)
        benchmark_vol = np.std(results['returns'][1:]) * np.sqrt(252)
        
        # Sharpe ratio (assuming 0 risk-free rate)
        sharpe = annual_return / volatility if volatility > 0 else 0
        benchmark_sharpe = benchmark_annual / benchmark_vol if benchmark_vol > 0 else 0
        
        # Maximum drawdown
        rolling_max = np.maximum.accumulate(equity)
        drawdown = (equity - rolling_max) / rolling_max
        max_drawdown = np.min(drawdown)
        
        # Calmar ratio
        calmar = annual_return / abs(max_drawdown) if max_drawdown != 0 else 0
        
        # Win rate
        winning_trades = np.sum(strategy_returns > 0)
        total_trades = np.sum(strategy_returns != 0)
        win_rate = winning_trades / total_trades if total_trades > 0 else 0
        
        # Profit factor
        gross_profit = np.sum(strategy_returns[strategy_returns > 0])
        gross_loss = abs(np.sum(strategy_returns[strategy_returns < 0]))
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else np.inf
        
        metrics = {
            'Total Return': f"{total_return:.2%}",
            'Benchmark Return': f"{benchmark_return:.2%}",
            'Annual Return': f"{annual_return:.2%}",
            'Benchmark Annual': f"{benchmark_annual:.2%}",
            'Volatility': f"{volatility:.2%}",
            'Sharpe Ratio': f"{sharpe:.2f}",
            'Benchmark Sharpe': f"{benchmark_sharpe:.2f}",
            'Max Drawdown': f"{max_drawdown:.2%}",
            'Calmar Ratio': f"{calmar:.2f}",
            'Win Rate': f"{win_rate:.2%}",
            'Profit Factor': f"{profit_factor:.2f}",
            'Total Trades': total_trades
        }
        
        return metrics, drawdown

In [ ]:
# Run backtest on test data
backtester = TradingBacktester(initial_capital=100000, transaction_cost=0.001)

# Generate signals
signals, probabilities = backtester.generate_signals(
    model_indicators, 
    X_test_ind, 
    threshold=0.1  # Only trade when confidence > 55% or < 45%
)

# Get actual returns for test period
test_start_idx = int(len(indicators_df) * 0.8) + 20  # Account for sequence length
test_returns = indicators_df['returns'].values[test_start_idx:test_start_idx + len(signals)]

# Run backtest
results = backtester.run_backtest(signals, test_returns, probabilities)

# Calculate metrics
metrics, drawdown = backtester.calculate_metrics(results)

print("="*50)
print("BACKTEST RESULTS")
print("="*50)
for key, value in metrics.items():
    print(f"{key:20}: {value}")

In [ ]:
# Visualize backtest results
fig, axes = plt.subplots(4, 1, figsize=(14, 16))

# 1. Equity curves
ax1 = axes[0]
ax1.plot(results['equity'], label='Strategy', linewidth=2, color='blue')
ax1.plot(results['benchmark_equity'], label='Buy & Hold', linewidth=2, color='gray', alpha=0.7)
ax1.set_title('Equity Curves', fontsize=14)
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Drawdown
ax2 = axes[1]
ax2.fill_between(range(len(drawdown)), drawdown, 0, color='red', alpha=0.3)
ax2.plot(drawdown, color='red', linewidth=1)
ax2.set_title('Drawdown', fontsize=14)
ax2.set_ylabel('Drawdown')
ax2.grid(True, alpha=0.3)

# 3. Signals and positions
ax3 = axes[2]
ax3.plot(results['positions'], label='Position', color='green', alpha=0.7)
ax3.axhline(y=0, color='black', linestyle='--', alpha=0.3)
ax3.set_title('Trading Positions', fontsize=14)
ax3.set_ylabel('Position Size')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Prediction probabilities
ax4 = axes[3]
ax4.plot(probabilities, label='Prediction Probability', color='purple', alpha=0.7)
ax4.axhline(y=0.5, color='black', linestyle='--', alpha=0.3)
ax4.axhline(y=0.55, color='green', linestyle='--', alpha=0.3, label='Long Threshold')
ax4.axhline(y=0.45, color='red', linestyle='--', alpha=0.3, label='Short Threshold')
ax4.set_title('Model Predictions', fontsize=14)
ax4.set_ylabel('Probability')
ax4.set_xlabel('Trading Day')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Monthly returns heatmap
def create_monthly_returns(strategy_returns, start_date):
    """
    Create monthly returns DataFrame for heatmap.
    """
    # Create date index
    dates = pd.date_range(start=start_date, periods=len(strategy_returns), freq='B')
    returns_series = pd.Series(strategy_returns, index=dates)
    
    # Resample to monthly
    monthly_returns = returns_series.resample('M').apply(lambda x: (1 + x).prod() - 1)
    
    # Create pivot table
    monthly_df = pd.DataFrame({
        'Year': monthly_returns.index.year,
        'Month': monthly_returns.index.month,
        'Return': monthly_returns.values
    })
    
    pivot = monthly_df.pivot(index='Year', columns='Month', values='Return')
    pivot.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'][:len(pivot.columns)]
    
    return pivot

# Get start date for test period
test_start_date = indicators_df.index[test_start_idx]
monthly_returns_pivot = create_monthly_returns(results['strategy_returns'], test_start_date)

# Plot heatmap
plt.figure(figsize=(14, 6))
sns.heatmap(monthly_returns_pivot, annot=True, fmt='.1%', cmap='RdYlGn', center=0,
            linewidths=0.5, cbar_kws={'label': 'Return'})
plt.title('Monthly Returns Heatmap', fontsize=14)
plt.ylabel('Year')
plt.xlabel('Month')
plt.tight_layout()
plt.show()

In [ ]:
# Return distribution analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Strategy returns distribution
ax1 = axes[0]
ax1.hist(results['strategy_returns'], bins=50, alpha=0.7, color='blue', label='Strategy', density=True)
ax1.hist(results['returns'][1:], bins=50, alpha=0.5, color='gray', label='Buy & Hold', density=True)
ax1.axvline(x=0, color='black', linestyle='--')
ax1.set_title('Return Distribution', fontsize=14)
ax1.set_xlabel('Daily Return')
ax1.set_ylabel('Density')
ax1.legend()

# Rolling Sharpe ratio
ax2 = axes[1]
rolling_sharpe = pd.Series(results['strategy_returns']).rolling(63).apply(
    lambda x: np.mean(x) / np.std(x) * np.sqrt(252) if np.std(x) > 0 else 0
)
ax2.plot(rolling_sharpe, color='green', label='Rolling Sharpe (63-day)')
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.3)
ax2.axhline(y=1, color='green', linestyle='--', alpha=0.3, label='Sharpe = 1')
ax2.set_title('Rolling Sharpe Ratio', fontsize=14)
ax2.set_xlabel('Trading Day')
ax2.set_ylabel('Sharpe Ratio')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Trade analysis
def analyze_trades(signals, returns, probabilities):
    """
    Analyze individual trades.
    """
    trades = []
    
    in_trade = False
    trade_start = 0
    trade_signal = 0
    
    for i in range(len(signals)):
        if not in_trade and signals[i] != 0:
            # Enter trade
            in_trade = True
            trade_start = i
            trade_signal = signals[i]
        elif in_trade and (signals[i] == 0 or signals[i] != trade_signal or i == len(signals)-1):
            # Exit trade
            trade_return = np.sum(trade_signal * returns[trade_start+1:i+1])
            trades.append({
                'start': trade_start,
                'end': i,
                'duration': i - trade_start,
                'direction': 'Long' if trade_signal > 0 else 'Short',
                'return': trade_return,
                'confidence': np.mean(probabilities[trade_start:i])
            })
            in_trade = False
            
            # Check if new trade starts
            if signals[i] != 0:
                in_trade = True
                trade_start = i
                trade_signal = signals[i]
    
    return pd.DataFrame(trades)

trades_df = analyze_trades(signals, test_returns, probabilities)

if len(trades_df) > 0:
    print(f"Total Trades: {len(trades_df)}")
    print(f"\nTrade Statistics:")
    print(trades_df[['duration', 'return']].describe())
    
    print(f"\nBy Direction:")
    print(trades_df.groupby('direction')['return'].agg(['count', 'mean', 'sum']))
else:
    print("No trades generated")

---
<a id='6-summary'></a>
## 6. Summary and Key Takeaways

### What We Learned

#### 1. Sequence Classification for Regime Detection
- LSTMs can classify market states (Bull, Bear, Consolidation, High Volatility)
- Rolling statistics help define regime labels
- Multi-class classification using CrossEntropyLoss

#### 2. Many-to-One vs Many-to-Many Architectures
- **Many-to-One**: Sequence → Single output (classification, next-day prediction)
- **Many-to-Many**: Sequence → Sequence (multi-step forecasting)
- Encoder-decoder architecture for sequence-to-sequence tasks
- Teacher forcing for training sequence generators

#### 3. LSTM + Technical Indicators
- Combining domain knowledge (indicators) with deep learning
- Attention mechanism highlights important time steps
- Bidirectional LSTM captures patterns in both directions

#### 4. Trading Strategy Backtest
- Position sizing based on model confidence
- Transaction costs significantly impact returns
- Multiple metrics needed: Sharpe, Calmar, Win Rate, Profit Factor
- Visualizations help understand strategy behavior

### Important Caveats

⚠️ **This is for educational purposes only!**

1. **Overfitting Risk**: Neural networks easily overfit to historical patterns
2. **Look-ahead Bias**: Ensure features don't use future information
3. **Regime Changes**: Models trained on past regimes may fail in new ones
4. **Transaction Costs**: Real costs may be higher than simulated
5. **Slippage**: Not modeled here but significant for high-frequency strategies
6. **Market Impact**: Large positions can move prices

### Next Steps
- Implement walk-forward optimization
- Add more sophisticated risk management
- Test on multiple assets
- Combine with fundamental data
- Explore transformer architectures

In [ ]:
# Final summary statistics
print("="*60)
print("FINAL SUMMARY")
print("="*60)

print("\n1. REGIME CLASSIFIER")
print(f"   - Test Accuracy: {test_accuracies[-1]:.2%}")
print(f"   - Number of Regimes: 4")

print("\n2. MANY-TO-MANY FORECASTER")
print(f"   - Forecast Horizon: {FORECAST_HORIZON} days")
print(f"   - Final Training Loss: {losses[-1]:.6f}")

print("\n3. LSTM + INDICATORS TRADING MODEL")
print(f"   - Features: {len(feature_cols)}")
print(f"   - Test Accuracy: {test_accuracies_ind[-1]:.2%}")

print("\n4. BACKTEST RESULTS")
for key in ['Total Return', 'Sharpe Ratio', 'Max Drawdown', 'Win Rate']:
    print(f"   - {key}: {metrics[key]}")

print("\n" + "="*60)
print("Notebook completed successfully!")
print("="*60)